<a href="https://colab.research.google.com/github/e40125/Breeze-Experiment/blob/main/breeze_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch datasets huggingface_hub -qqqU

In [ ]:
!pip install trl accelerate bitsandbytes peft -qqqU

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# **Finetune configurations**

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id="MediaTek-Research/Breeze-7B-Instruct-v1_0"

def get_model_and_tokenizer(model_id):
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = 'right'
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="bfloat16", bnb_4bit_use_double_quant=False
  )
  model = AutoModelForCausalLM.from_pretrained(
      model_id, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.bfloat16
  )
  return model, tokenizer

model, tokenizer = get_model_and_tokenizer(model_id)

from datasets import load_dataset
dataset = load_dataset("ethanchuangg/ckd100")

def format_qa(example):
    sys_prompt = "你是一個專業的醫療護理師，專門回答有關慢性腎臟疾病的問題。請回答以下問題："
    formatted = f"<s>{sys_prompt} [INST] {example['question']} [/INST] {example['answer']}"
    return {"text": formatted}
formatted_dataset = dataset['train'].map(format_qa)

from transformers import TrainingArguments
from trl import SFTTrainer, SFTConfig

##########################################################################################
#Tune Hyperparameters
output_model="b003"
learning_rate=2e-5
batch_size=4
epochs=4
##########################################################################################

from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
peft_config = LoraConfig(
        r=64, lora_alpha=32, lora_dropout=0.1, bias="none", task_type="CAUSAL_LM"
    )
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=batch_size,
        learning_rate=2e-5,
        # warmup_ratio=0.1,
        lr_scheduler_type="constant",
        logging_steps=10,
        num_train_epochs=epochs,
        # max_steps=250,
        bf16=True,
    )
trainer = SFTTrainer(
        model=model,
        train_dataset=formatted_dataset,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=1024
    )
trainer.train()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version

Step,Training Loss
10,2.562000
20,2.205400
30,1.850800
40,1.624900
50,1.536100
60,1.392100
70,1.284500
80,1.289100
90,1.192200
100,1.190000


TrainOutput(global_step=100, training_loss=1.612713975906372, metrics={'train_runtime': 182.4074, 'train_samples_per_second': 2.193, 'train_steps_per_second': 0.548, 'total_flos': 2992647729905664.0, 'train_loss': 1.612713975906372, 'epoch': 4.0})

In [ ]:
trainer.push_to_hub(
    output_model
)

events.out.tfevents.1725011082.35a212146df8.10100.0:   0%|          | 0.00/8.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ethanchuangg/b003/commit/fc4b8b1cf63a940a86863b4ba894b0498d244579', commit_message='b003', commit_description='', oid='fc4b8b1cf63a940a86863b4ba894b0498d244579', pr_url=None, pr_revision=None, pr_num=None)

# **Load Base/Finetuned Model**

In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import PeftModel, PeftConfig

# Load the base model config
base_model_id = "MediaTek-Research/Breeze-7B-Instruct-v1_0"

# Set up quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False
)

# Load base model with quantization
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# Load PEFT config #uncomment to load peft finetuned model
# peft_model_id = output_model
# peft_config = PeftConfig.from_pretrained(peft_model_id)

# # Load the PEFT model
# testmodel = PeftModel.from_pretrained(base_model, peft_model_id)

# Get tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
# Set pad token
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# **Set up questions and see responses**

In [ ]:
q1='飲食如何影響慢性腎臟病？'
q2='如何控制血磷水平？'
q3='飲食對慢性腎臟病有什麼影響？'
q4='如何有效管理血磷水平？'
q5='我覺得慢性腎臟病惡化怎麼辦?'
q6='好難控制磷的攝取量，有什麼方法呢?'


In [ ]:
questions = [q1, q2, q3, q4, q5, q6]

all_outputs = []

for q in questions:
    chat = [
        {"role": "system", "content": "你是一個專業的醫療護理師，專門回答有關慢性腎臟疾病的問題。"},
        {"role": "user", "content": q}
    ]

    inputs = tokenizer.apply_chat_template(chat, return_tensors="pt")
    input_ids = inputs.to(basemodel.device)   #or testmodel.device for finetuned model

    attention_mask = torch.ones_like(input_ids)

    outputs = basemodel.generate(             #or testmodel.generate for finetuned model
        input_ids=input_ids,
        attention_mask=attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=256,
        # repetition_penalty=1.2
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_text = response.split('[/INST]')[-1].strip()
    all_outputs.append(generated_text)

for i, output in enumerate(all_outputs, 1):
    print(f"Question {i}:")
    print(output)
    print("\n")

Question 1:
飲食對慢性腎臟病的影響是巨大的，因為腎臟是負責過濾血液中的廢物，包括食物中的蛋白質、碳水化合物、脂肪和其他物質。如果飲食不健康，腎臟需要工作得更加努力，從而增加腎臟的負擔。以下是一些飲食習慣如何影響慢性腎臟病：

1. 高血壓：高血壓是導致慢性腎臟病的一個主要因素。高鹽飲食會增加血壓，因此減少鹽分攝入是重要的。
2. 高蛋白質攝入：腎臟需要過濾血液中的廢物，包括蛋白質。過多的蛋白質攝入會增加腎臟的負擔，從而增加慢性腎臟病的風險。
3. 高鉀飲食：鉀是一種重要的礦物質，但腎臟病患者需要限制鉀的攝入，因為過多的鉀會導致高鉀血症，從而增加腎臟的負擔。
4. 高磷飲食：磷是另一種重要的礦物質，但腎臟病患者需要限制磷的攝入，因為過多的磷會導致高磷血症，從而增加腎臟的負擔。
5. 高脂肪攝入：過多的脂肪會增加腎臟的負擔，從而增加慢性腎臟病的風險。
6.


Question 2:
控制血磷水平的關鍵是限制磷的攝入量，同時保持良好的鈣質平衡。以下是一些建議：

1. 限制磷的攝入量：
   - 限制高磷食物的攝入，如動物內臟、豆製品、深綠色蔬菜、全穀類、巧克力、碳酸飲料等。
   - 選擇低磷食物，如白面、白麪包、白麪粉、白薯、白糖、牛奶、乳酪、雞蛋、瘦肉、家禽肉、魚類、蔬菜、水果等。
   - 注意食品包裝上的營養成分表，選擇磷含量較低的食品。

2. 合理補充鈣：
   - 每天補充適量的鈣，以維持血鈣平衡。鈣的攝入量應根據個人的血磷水平、腎功能、年齡等因素而定。
   - 鈣的來源包括牛奶、乳酪、乳酪製品、深綠色蔬菜、堅果、小魚干、小魚乾、海魚、海蜇等。

3. 控制維生素D的攝入：
   - 維生素D有助於鈣的吸收，但過量攝入可能導致血磷水平升高。因此，應在


Question 3:
飲食對慢性腎臟病有重大的影響。在慢性腎臟病的患者中，飲食控制是一個非常重要的方面。以下是一些飲食方面的建議：

1. 限制蛋白質的攝入量：腎臟病患者需要限制蛋白質的攝入量，因為過多的蛋白質會增加腎臟的負擔。建議每天攝入蛋白質在每公斤體重 0.6-0.8 克，例如，一個 60 公斤的患者，每天蛋白質攝入量在 36-48 克。

2. 限制鈉的攝入量：鈉是一種礦物質，過多的鈉攝入會導致高血壓，增加腎臟的負擔。建議每天鈉的攝入量在 2-3 克。

3. 限制磷的攝入量：磷